In [1]:
import numpy as np
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from data import load_dataset

## 要求
实现一个学习算法的整体结构
* 获取并定义模型输入
* 初始化参数
* 计算成本函数及其梯度
* 使用优化算法（梯度下降）
    * 循环：
    * 计算当前损失（正向传播）
    * 计算当前梯度（反向传播）
    * 更新参数（梯度下降）

请实现相关函数，其中读取数据不需要实现。并在主模型函数中编写逻辑。

### 读取数据
1、按照向量化伪代码实现的形状要求将样本数据进行转换
2、标准化数据处理

In [9]:
train_x, train_y, test_x, test_y, classes = load_dataset()
print ("训练集的样本数: ", train_x.shape[0])
print ("测试集的样本数: " , test_x.shape[0])
print ("train_x形状: ", train_x.shape)
print ("train_y形状: ", train_y.shape)
print ("test_x形状: ", test_x.shape)
print ("test_x形状: ", test_y.shape)

训练集的样本数:  209
测试集的样本数:  50
train_x形状:  (209, 64, 64, 3)
train_y形状:  (1, 209)
test_x形状:  (50, 64, 64, 3)
test_x形状:  (1, 50)


In [7]:
train_y

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [8]:
train_x

array([[[[ 17,  31,  56],
         [ 22,  33,  59],
         [ 25,  35,  62],
         ...,
         [  1,  28,  57],
         [  1,  26,  56],
         [  1,  22,  51]],

        [[ 25,  36,  62],
         [ 28,  38,  64],
         [ 30,  40,  67],
         ...,
         [  1,  27,  56],
         [  1,  25,  55],
         [  2,  21,  51]],

        [[ 32,  40,  67],
         [ 34,  42,  69],
         [ 35,  42,  70],
         ...,
         [  1,  25,  55],
         [  0,  24,  54],
         [  1,  21,  51]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
        

In [10]:
train_x = train_x.reshape(train_x.shape[0], -1).T
test_x = test_x.reshape(test_x.shape[0], -1).T
print(train_x.shape, test_x.shape)

(12288, 209) (12288, 50)


In [11]:
train_x = train_x/255.
test_x = test_x/255.

In [12]:
def basic_sigmoid(x):
    """
    计算sigmoid函数
    """
    
    ### 开始
    s = 1 / (1 + np.exp(-x))
    ### 结束
    
    return s

### 算法结构实现
分别构建算法的不同模块
* 初始化参数
* 计算成本函数及其梯度
* 使用优化算法（梯度下降）
    * 循环：
    * 计算当前损失（正向传播）
    * 计算当前梯度（反向传播）
    * 更新参数（梯度下降）
 
### 初始化参数（假设有 n 个特征，m 个样本）

```python
J = 0
W = np.zeros([n, 1])
b = 0
```

---

### 激活函数定义（以 Sigmoid 为例）$\sigma(z) = \frac{1}{1 + e^{-z}}$

---

### 前向传播 计算出梯度与损失

$Z = W^T X + b$
               (1,m)

$A = \sigma(Z)$                             (1,m)

### 计算损失：$L(\hat{y},y)=−(ylog\hat{y})−(1−y)log(1−\hat{y})$

### 反向传播（计算参数的更新梯度）
$dZ = A - Y$                    (1,m)


$dW = \frac{1}{m} X dZ^T$           (n,m) * (m,1) = (n,1)


$db = \frac{1}{m} \sum dZ$

---

### 参数更新（使用学习率 ($\alpha$)）
$W := W - \alpha dW$


$b := b - \alpha db$

### 初始化网络参数

In [13]:
def initialize_with_zeros(shape):
    """
    创建一个形状为 (shape, 1) 的w参数和b=0.
    return:w, b
    """
    
    ### 开始
    w = np.zeros((shape, 1))
    b = 0

    ### 结束

    assert(w.shape == (shape, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

### 前向和反向传播
根据损失函数、前后传播向量化代码

In [15]:
def propagate(w, b, X, Y):
    """
    参数：w,b,X,Y：网络参数和数据
    Return:
    损失cost、参数W的梯度dw、参数b的梯度db
    """
    # 获取样本数
    m = X.shape[1]
    
    # 前向传播
    ### 开始
    # w (n,1), x (n, m)
    # np.dot(w.T, X)  # (1, m)
    A = basic_sigmoid(np.dot(w.T, X) + b)

    #计算损失
    #损失计算方法-(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    #由于计算的是所有损失,所以需要除以样本数m
    cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    ### 结束

    # 反向传播
    ### 开始
    dz = A - Y
    dw = 1 / m * np.dot(X, dz.T)
    db = 1 / m * np.sum(dz)
    ### 结束

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

### 优化过程
实现优化函数. 全局的参数随着 $w$  $b$ 对损失 $J$进行优化改变. 对参数 $\theta$实行 $ \theta = \theta - \alpha \text{ } d\theta$, 并制定 $\alpha$ 学习率。

In [16]:
def optimize(w, b, X, Y, num_iterations, learning_rate):
    """
    参数：
    w:权重,b:偏置,X特征,Y目标值,num_iterations总迭代次数,learning_rate学习率
    Returns:
    params:更新后的参数字典
    grads:梯度
    costs:损失结果
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        # 梯度更新计算函数
        ### 开始
        grads, cost = propagate(w, b, X, Y)
        
        # 取出两个部分参数的梯度
        dw = grads['dw']
        db = grads['db']
        
        # 按照梯度下降公式去计算
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        ### 结束
        
        if i % 100 == 0:
            costs.append(cost)
        if i % 100 == 0:
            print("损失结果 %i: %f" %(i, cost))
            print(b)
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

### 预测函数（不用实现）
利用得出的参数来进行测试得出准确率

In [17]:
def predict(w, b, X):
    '''
    利用训练好的参数预测
    
    return：预测结果
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # 计算结果
    ### 开始
    A = basic_sigmoid(np.dot(w.T, X) + b)
    ### 结束

    for i in range(A.shape[1]):
        
        ### 开始
        if A[0, i] <= 0.5:
            Y_prediction[0, i] = 0
        else:
            Y_prediction[0, i] = 1
        ### 结束
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

### 整体逻辑实现

In [18]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5):
    """
    """
    
    ### 开始
    
    # 初始化参数
    w, b = initialize_with_zeros(X_train.shape[0])

    # 梯度下降
    # params:更新后的网络参数
    # grads:最后一次梯度
    # costs:每次更新的损失列表
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)
    
    # 获取训练的参数
    # 预测结果
    w = params['w']
    b = params['b']
    
    Y_prediction_train = predict(w, b, X_train)
    Y_prediction_test = predict(w, b, X_test)
    ### 结束

    # 打印准确率
    print("训练集准确率: {} ".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("测试集准确率: {} ".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

### 测试

In [19]:
d = model(train_x, train_y, test_x, test_y, num_iterations = 2000, learning_rate = 0.005)

损失结果 0: 0.693147
-0.000777511961722488
损失结果 100: 0.584508
-0.0043827623417682
损失结果 200: 0.466949
-0.006796745374030197
损失结果 300: 0.376007
-0.00896621604504307
损失结果 400: 0.331463
-0.010796335272035095
损失结果 500: 0.303273
-0.012282447313396536
损失结果 600: 0.279880
-0.013402386273819074
损失结果 700: 0.260042
-0.014245091216970826
损失结果 800: 0.242941
-0.014875420165524861
损失结果 900: 0.228004
-0.015341288386626656
损失结果 1000: 0.214820
-0.01567878837544241
损失结果 1100: 0.203078
-0.015915536343924584
损失结果 1200: 0.192544
-0.01607292624287495
损失结果 1300: 0.183033
-0.016167692508505714
损失结果 1400: 0.174399
-0.016213022073676538
损失结果 1500: 0.166521
-0.016219364232163885
损失结果 1600: 0.159305
-0.016195032712389285
损失结果 1700: 0.152667
-0.016146661324349915
损失结果 1800: 0.146542
-0.01607955397736278
损失结果 1900: 0.140872
-0.015997956805040348
训练集准确率: 99.04306220095694 
测试集准确率: 70.0 
